<a href="https://colab.research.google.com/github/mwahaha-umich/ACLUFinalProject/blob/main/ACLU_Project_Normalize_Data_Part_2_(Transform).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
from os import listdir
from os.path import isfile, join

GoogleDriveBase = "/content/drive/My Drive/"
WorkingDirectory = "Projects/ACLU/"
BasePickeDrive = GoogleDriveBase + WorkingDirectory + "Pickle/"
WorkingFiles = GoogleDriveBase + WorkingDirectory + "WorkingFiles/"
ListOfAllRequiredFiles = [WorkingDirectory + 'LetPeopleVote Data/LPVCall120210621-8755504337.xls', 
                          WorkingDirectory + 'LetPeopleVote Data/LPVMail120210621-11078584334.xls',
                          WorkingDirectory + 'LetPeopleVote Data/LPVMail220210621-13283358728.xls']
for PathToFile in ListOfAllRequiredFiles:
  path, file = os.path.split(PathToFile)

  #Break the folders out into a list
  folders = path.split(os.sep)

  RunningPath = GoogleDriveBase
  for folder in folders:
    RunningPath = RunningPath + folder + "/"
    try:
      FilesAndFolders = [f for f in listdir(RunningPath) ]
    except:
      print("The following path does not exist. Please create it before continuing.")
      print(RunningPath)

  if(os.path.exists(GoogleDriveBase + PathToFile) == False):
      print("The following file does not exist. Please create it before continuing.")
      print(GoogleDriveBase + PathToFile)

In [ ]:
import numpy
import sys
import csv
import string
import pandas as pd
import gc
import re
import string

In [ ]:
#dfVoters = pd.read_csv(WorkingFiles + "dfVoters.csv", dtype=str) #Not needed for the transformations
dfMerged = pd.read_csv(WorkingFiles + "dfMerged.csv")


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,17,18,19,20,21,23,27,29,31,32,34,36,39,40,42,44,46,48,49,50,52,53,54,55,56,57,58,60,61,63,65,66,67,68,70,71,72,73,74,75,76,77,78,79,80,82,83,85,86,87) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


This is an example of duplicate data we found by combining the data files. It apepars that if a person is included in multiple outreach attempts, their full data will be included in each outreach attempt email. 

We went back and verified that this was expected behavior with the UCLA. Just make a note of it for now, we will be actually removing it further into this notebook.

In [ ]:
dfMerged[dfMerged["Voter File VANID"] == 11]

,Unnamed: 0,index,Voter File VANID,StateFileID,General20,General19,General18,General17,General16,General15,General14,General12,General11,General10,General09,General08,General06,General04,General02,General00,General98,Municipal13,PresPrimary20,PresPrimary20Party,PresPrimary16,PresPrimary16Party,PresPrimary12,PresPrimary12Party,PresPrimary08,PresPrimary08Party,Primary20,Primary20Party,Primary19Party,Primary18,Primary18Party,Primary17,Primary17Party,Primary16,Primary16Party,Primary15,...,Primary09Party,Primary08,Primary08Party,Primary06,Primary04,Primary02,Primary01,Primary00,Primary98,Special20,Special19,Special18,Special17,Special16,Special15,Special13,Special12,Special11,Special10,Special09,Special08,Special07,Special06,Special05,Special04,Special03,Special02,Special01,Special00,Special99,Special98,2020_Biden_Support,MarijuanaConviction,PoliceAccountability,Absentee_Voting,Ballot_Application,VBM_Application,Voting_Aug_Prim,Voting_Nov_Gen,VotedStatus
611443,611443,76968,11,106369799.0,A,NaN,P,NaN,P,NaN,NaN,NaN,NaN,NaN,NaN,P,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1235993,1235993,42602,11,106369799.0,A,NaN,P,NaN,P,NaN,NaN,NaN,NaN,NaN,NaN,P,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


A: Absentee

P: Polls (if there is a D or R, that is specifying which party they voted during the primary)

M: This one is new to me. Let me check in with our database administrator

Y: Yes—indicates that they voted, but we don’t have data on how they voted (in person vs. absentee)

In [ ]:
primary_key = ["Voter File VANID"]
columns_processed = []

def ReshapeDataframeYears(df):
  ColumnsToTransform = ["Special##", "General##", "Primary##", "PresPrimary##", "PresPrimary##Party", "Primary##Party", "Municipal##"]
  #ColumnsToTransform = ["Special##"]
  melted_dataframes = []
  for name in ColumnsToTransform:
    ColumnsForName = []
    years = []
    number_index = name.index("##")
    for column in df.columns:
      if len(column) < len(name):
        continue
      first = column[0:number_index]
      name_first = name[0:number_index]
      number = column[number_index:number_index+2]
      if len(column) - number_index+2 == 0:
        second = ""
        name_second = ""
      else:
        second = column[number_index+2:len(column)]
        name_second = name[number_index+2:len(name)]

      if first == name_first and second == name_second:
        ColumnsForName.append(column)
        columns_processed.append(column)
        year = column[len(first):len(first)+2]

        if int(year) < 50:
          year = "20" + year
        else:
          year = "19" + year
        years.append(year)
        #print(int(year))
    df_reduced = df[primary_key + ColumnsForName]
    df_reduced.columns = primary_key + years
    df_melted = pd.melt(df_reduced, id_vars=primary_key, value_vars=years)
    
    #Restructure the columns
    final_columns = primary_key.copy()
    final_columns.append("year")
    final_columns.append("participation")
    df_melted.columns = final_columns
    df_melted["electiontype"] = name
    #Drop records where we don't have participation data. 
    df_melted = df_melted[df_melted['participation'].notna()] 
    #print(df_melted.head())
    melted_dataframes.append(df_melted)
  return melted_dataframes

def CombineMeltedDataframes(melted_dataframes):
  #Now join them all back together
  #Rather silly that this is my method, but I tried a few different versions to find the most memory efficient way to do this
  return pd.concat(melted_dataframes)

melteddfs = ReshapeDataframeYears(dfMerged)
ColumnsThatWereNotPartOfTheRestructuredColumns = set(dfMerged.columns) - set(columns_processed)


The following cell is just here fora  visual check to ensure that we aren't going to be leaving beyind any columns that we should care about. 

In [ ]:
for column in ColumnsThatWereNotPartOfTheRestructuredColumns:
  print(column)
  print(dfMerged[column].unique())

dfMerged[ColumnsThatWereNotPartOfTheRestructuredColumns].head()

StateFileID
[4.22800000e+05 3.37528230e+07 5.00800000e+05 ... 4.94710000e+04
 1.99367000e+05 1.05443025e+08]
Voting_Nov_Gen 
[nan 'Yes' 'Maybe/Unsure' 'No']
Voting_Aug_Prim
[nan 'Yes' 'No' 'Maybe/Unsure']
Voter File VANID
[  144325    95301   186517 ...    12856    31452 10108394]
PoliceAccountability
[nan 'Yes' 'Other/Unknown' 'Maybe' 'No']
Absentee_Voting
[nan 'Yes' 'EDay/Early at Clerks' 'Other (Note)' 'No' 'Needs AV ballot'
 'Not Voting']
2020_Biden_Support
[nan '5. Strong Oppose' '1. Strong Support' 'Undecided/Unsure'
 '4. Lean Oppose' '2. Lean Support' 'Not Voting']
VotedStatus
[nan]
Ballot_Application
[nan 'No - voting at polls' 'No - need help/info' 'Yes' 'Not Asked']
index
[     0      1      2 ... 343455 343456 343457]
MarijuanaConviction
[nan 'Yes' 'No' 'Other/Unknown']
Unnamed: 0
[      0       1       2 ... 3196828 3196829 3196830]
VBM_Application
[nan 'Yes' 'No' 'Other (Note)' 'Already PermAV List']


,StateFileID,Voting_Nov_Gen,Voting_Aug_Prim,Voter File VANID,PoliceAccountability,Absentee_Voting,2020_Biden_Support,VotedStatus,Ballot_Application,index,MarijuanaConviction,Unnamed: 0,VBM_Application
0,422800.0,NaN,NaN,144325,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN
1,33752823.0,NaN,NaN,95301,NaN,NaN,NaN,NaN,NaN,1,NaN,1,NaN
2,500800.0,NaN,NaN,186517,NaN,NaN,NaN,NaN,NaN,2,NaN,2,NaN
3,265552.0,NaN,NaN,61006,NaN,NaN,NaN,NaN,NaN,3,NaN,3,NaN
4,226538.0,NaN,NaN,42852,NaN,NaN,NaN,NaN,NaN,4,NaN,4,NaN


In [ ]:
del dfMerged
gc.collect()

dfreshaped = CombineMeltedDataframes(melteddfs)

del melteddfs
gc.collect()

0

In [ ]:
dfreshaped.head()

,Voter File VANID,year,participation,electiontype
0,144325,2020,A,Special##
2,186517,2020,P,Special##
8,126306,2020,P,Special##
10,127054,2020,A,Special##
11,157007,2020,P,Special##


In [ ]:
dfreshapedUnique = dfreshaped.drop_duplicates()

In [ ]:
dfreshapedUnique = dfreshapedUnique.sort_values(["Voter File VANID", "year"])

In [ ]:
dfreshapedUnique.to_csv(WorkingFiles + "dfreshapedUnique.csv", encoding='utf-8')

In [ ]:
import altair as alt

df_summary = dfreshaped.groupby(["year", "electiontype"]).count().reset_index()
alt.Chart(df_summary).mark_line().encode(
    x='year',
    y='participation',
    color='electiontype',
)


#df_summary = df_reshaped.groupby(["year"]).count().reset_index()
#ax = plt.gca()
#df_summary.plot(kind='line',x='year',y='Special##',ax=ax)
#df_summary.head()

alt.Chart(...)